In [19]:
import warnings
warnings.filterwarnings('ignore')

import os
import cv2
import csv
import pickle
import pydicom
import numpy as np
import pandas as pd 
from glob import glob


# import the necessary packages
# import keras
# import tensorflow as tf
# from keras import backend as K

from dataset import prepare_data
from generator import DataGenerator, label_generator
# from metric_loss import my_iou_metric, iou_metric_batch_val, bce_dice_loss
# from predict import predict_result_val, prepare_test, get_test, get_prediction, get_rles

# import seg_models
# keras.backend.set_image_data_format('channels_last')

# from keras.optimizers import SGD
# from keras.callbacks import ModelCheckpoint

import sys
sys.path.insert(0, 'siim-acr-pneumothorax-segmentation')
from mask_functions import rle2mask, mask2rle

## Seeding 
seed = 1994
np.random.seed = seed
os.environ['PYTHONHASHSEED'] = str(seed)
# tf.seed = seed

import gc   #Gabage collector for cleaning deleted data from memory

# Dataset

In [42]:
# defining configuration parameters
org_size = 1024 # original image size
img_size = 512  # image resize size
batch_size = 10 # batch size for training unet

## Load train and validation data from files

In [43]:
pkl_file_train = open('process_data/X_train.pkl', 'rb')

X_train = pickle.load(pkl_file_train)

In [44]:
pkl_file_val = open('process_data/X_val.pkl', 'rb')

X_val = pickle.load(pkl_file_val)

In [45]:
pkl_file_masks = open('process_data/masks.pkl', 'rb')

masks = pickle.load(pkl_file_masks)

## Data generation & Augmentations

In [46]:
import albumentations as A

In [47]:
training_augmentation = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.OneOf([
        #A.CLAHE(),
        A.RandomContrast(),
        A.RandomGamma(),
        A.RandomBrightness(),
         ], p=0.3),
    A.OneOf([
        A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
        A.GridDistortion(),
        A.OpticalDistortion(distort_limit=2, shift_limit=0.5),
        ], p=0.3),
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=20,
                                        interpolation=cv2.INTER_LINEAR, border_mode=cv2.BORDER_CONSTANT, p=1),
    A.RandomSizedCrop(min_max_height=(img_size, img_size), height=img_size, width=img_size,p=0.25)
],p=1)

In [48]:
params_train = {'img_size': img_size,
          'batch_size': batch_size,
          'n_channels': 3,
          'shuffle': True,
           'augmentations':training_augmentation,
           }

params_val = {'img_size': img_size,
          'batch_size': batch_size,
          'n_channels': 3,
          'shuffle': True,
         }

# Generators
training_generator = DataGenerator(X_train, masks, **params_train)
validation_generator = DataGenerator(X_val, masks, **params_val)

In [49]:
x, y = training_generator.__getitem__(0)
print(x.shape, y.shape)

(10, 512, 512, 3) (10, 512, 512, 1)
